# Network Analysis of Harry Potter Book Series

## Setup

In [ ]:
!pip install -r requirements.txt
from sentiment import *

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


## Function Definitions for Text Analysis

`happiness(doc)` takes a list of words and computes the average happiness score using the Hedonometer

`emotion_score(doc)` takes a list of words and computes a dictionary of average emotion scores among the emotions: _Anger, Anticipation,	Disgust, Fear, Joy, Sadness, Surprise_ and _Trust_

`vader_sentiment(doc)` takes a list of words and/or sentences and computes the average VADER compund score

`TF_IDF(docs_to_analyse, all_docs)` computes TF and TF-IDF score of terms in docs_to_analyse. all_docs are used to comput IDF scores 


([FreqDist({'love': 0.5, 'hate': 0.5}),
  FreqDist({'love': 0.4, 'hej': 0.2, 'med': 0.2, 'dig': 0.2})],
 [FreqDist({'hate': 0.34657359027997264, 'love': 0.0}),
  FreqDist({'hej': 0.13862943611198905, 'med': 0.13862943611198905, 'dig': 0.13862943611198905, 'love': 0.0})])

In [ ]:
import pandas as pd
import pickle as pkl
from clean_books import clean_book

In [ ]:
chapter_info = pd.read_pickle('data/chapter_dataframe.pkl')
chapter_info

,Global Chapter,Local Chapter,Title,Approx Story Time
0,1,1,The Boy Who Lived,1981-11-01
1,2,2,The Vanishing Glass,1991-06-23
2,3,3,The Letters from No One,1991-07-23
3,4,4,The Keeper of Keys,1991-07-31
4,5,5,Diagon Alley,1991-07-31
...,...,...,...,...
194,195,33,The Prince’s Tale,1998-05-02
195,196,34,The Forest Again,1998-05-02
196,197,35,King’s Cross,1998-05-02
197,198,36,The Flaw in the Plan,1998-05-02


In [ ]:
with open('data/characters_by_house.pkl', 'rb') as file:
    characters_by_communities = pkl.load(file)

characters_by_communities

{'Gryffindor': ['Albus Dumbledore',
  'Alicia Spinnet',
  'Andrew Kirke',
  'Angelina Johnson',
  'Bill Weasley',
  'Celestina Warbeck',
  'Charlie Weasley',
  'Cormac McLaggen',
  'Dean Thomas',
  'Demelza Robins',
  'Dennis Creevey',
  'Euan Abercrombie',
  'Fred Weasley',
  'Geoffrey Hooper',
  'George Weasley',
  'Ginny Weasley',
  'Godric Gryffindor',
  'Harry Potter',
  'Jack Sloper',
  'James Potter',
  'James Sirius Potter',
  'Jimmy Peakes',
  'Katie Bell',
  'Kenneth Towler',
  'Lavender Brown',
  'Lee Jordan',
  'Lily Potter',
  'Minerva McGonagall',
  'Natalie McDonald',
  'Nearly-Headless Nick',
  'Neville Longbottom',
  'Oliver Wood',
  'Panju Weasley',
  'Parvati Patil',
  'Patricia Stimpson',
  'Percy Weasley',
  'Peter Pettigrew',
  'Remus Lupin',
  'Ritchie Coote',
  'Romilda Vane',
  'Ron Weasley',
  'Rose Granger-Weasley',
  'Rubeus Hagrid',
  'Seamus Finnigan',
  'Sir Cadogan',
  'Vicky Frobisher',
  'Wormtail',
  'Yann Fredericks'],
 'Hufflepuff': ['Artemisia Lufk

In [ ]:

clean_book("data/Book 6 - The Half Blood Prince_djvu.txt.html")

'* J \n\n\n\n/ \n\n\n\n\nTHE OTHER MINISTER \n\nIt was nearing midnight and the Prime Minister was \nsitting alone in his office, reading a long memo that \nwas slipping through his brain without leaving the \nslightest trace of meaning behind. He was waiting for \na call from the President of a far distant country, and \nbetween wondering when the wretched man would \ntelephone, and trying to suppress unpleasant \nmemories of what had been a very long, tiring, and \ndifficult week, there was not much space in his head \nfor anything else. The more he attempted to focus on \nthe print on the page before him, the more clearly the \nPrime Minister could see the gloating face of one of \nhis political opponents. This particular opponent had \nappeared on the news that very day, not only to \nenumerate all the terrible things that had happened \nin the last week (as though anyone needed reminding) \nbut also to explain why each and every one of them \nwas the government’s fault. \n\nThe Pr

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=85fb65b4-b596-4730-837e-04e86eafe419' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>